# Setting environment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
!git clone https://github.com/utkarsh512/cdscraper.git

In [ ]:
%cd cdscraper/
%pip install .

In [ ]:
%cd ..

# Scraping profile metadata

In [ ]:
from thread import Thread, Comment
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
from copy import deepcopy

In [ ]:
comments = dict()

categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

categories_selected = ['politics2', 'religion', 'world', 'science', 'law', 'technology']
categories_labels = ['politics', 'religion', 'world', 'science', 'law', 'technology']

for x in categories:
    comments[x] = list()

In [ ]:
for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()
    print(f'{cat} - {len(threads)}')
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append(v)
            except:
                authors[v.author] = list()
                authors[v.author].append(v)
    ctr = 0
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            comments[cat].append(foo)
            ctr += 1

In [ ]:
users = set()

for cat in categories_selected:
    for comment in comments[cat]:
        users.add(comment['author'])

In [ ]:
len(users)

In [ ]:
with open('usernames.txt', 'w', encoding='utf-8') as f:
    for user in users:
        f.write(f"{user}\n")

In [ ]:
!cdscraper -i usernames.txt -o /content/gdrive/MyDrive/DL/CreateDebate/profile/results.json